In [2]:
import numpy as np
from pathlib import Path
import spatialdata_io
import spatialdata
import json
import time
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import concurrent.futures
from joblib import Parallel, delayed

import sys
sys.path.extend(['../../scripts','../../scripts/xenium'])
import coexpression
import readwrite
cfg = readwrite.config()

In [21]:
results_dir = Path(cfg['results_dir'])
figures_dir = Path(cfg['figures_dir'])
palette_dir = Path(cfg['xenium_metadata_dir'])

# Params
segmentation_palette = palette_dir / 'col_palette_segmentation.csv'

# Access the arguments
panel = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/results/resolvi_coexpression/10x_0um/breast/breast')
method = 'conditional'
target_count = 30
min_positivity_rate = 0.01
min_cond_coex=0.
min_cond_coex_mode='both'
cc_cutoff = 1.5
log = False
ref_segmentation = '10x_0um'
ref_oversegmentation = '10x_15um'
palette = pd.read_csv(segmentation_palette, index_col=0).iloc[:, 0]
showfliers = False

# vars
coexpression_dir = panel.parents[2]
plot_panel = panel.stem
plot_condition = panel.parents[0].stem
xenium_levels = ["segmentation", "condition", "panel", "donor", "sample"]
hue = "segmentation"
hue_order = ["10x_mm_0um", "10x_mm_5um", "10x_mm_15um", "10x_0um", "10x_5um", "10x_15um", "baysor", "proseg", "segger"]


# Read coexpression results
cc_paths = []
for segmentation in (segmentations := coexpression_dir.iterdir()):
    for condition in (conditions := segmentation.iterdir()):
        if condition.stem != plot_condition:
            continue
        for panel in (panels := condition.iterdir()):
            if panel.stem != plot_panel:
                continue
            for donor in (donors := panel.iterdir()):
                for sample in (samples := donor.iterdir()):
                    k = (
                        segmentation.stem,
                        condition.stem,
                        panel.stem,
                        donor.stem,
                        sample.stem,
                    )

                    cc_paths.append((k, method, target_count))

CC, pos_rate = readwrite.read_coexpression_files(cc_paths, coexpression_dir)


# automatically select min_cond_coex
if method in ["conditional", "jaccard"] and min_cond_coex == "auto":
    min_cond_coex = float(np.min([np.nanmin(CC[k][method, target_count].replace(0.0, np.nan)) for k in CC.keys()]))


# compute ratio with ref_segmentation
CCdiff = {}
spurious_gene_pairs = {}
for k in CC.keys():
    if k[0] == ref_segmentation:
        continue

    k_ref = (ref_segmentation, *k[1:])
    k_ref_over = (ref_oversegmentation, *k[1:])

    if k_ref not in CC.keys() or k_ref_over not in CC.keys():
        continue 

    CCdiff[k] = {}
    spurious_gene_pairs[k] = {}


    (CCdiff[k][method, target_count], spurious_gene_pairs[k][method, target_count]) = (
        coexpression.compare_segmentations(
            CC_ref_seg=CC[k_ref][method, target_count],
            CC_other_seg=CC[k][method, target_count],
            pos_rate_ref_seg=pos_rate[k_ref][method, target_count],
            pos_rate_other_seg=pos_rate[k][method, target_count],
            min_positivity_rate=min_positivity_rate,
            min_cond_coex=min_cond_coex,
            min_cond_coex_mode=min_cond_coex_mode,
            cc_cutoff=cc_cutoff,
            method=method,
        )
    )
